In [1]:
"""
Run ct calculation then dump 
the integral in fci dump
"""
from ct.rhf_energy import rhf_energy
from ct.ct import canonical_transform
import psi4
import numpy as np
from psi4.core import MintsHelper
from write_dump import write_dump
# set geometry and basis
psi_mol = psi4.geometry(
    """
    Li  0 0 0
    Li 0 0 r
    symmetry c1
    """
)
psi4.core.clean_options()
psi4.core.clean()
psi4.core.clean_variables()
psi4.set_output_file("ct_ucc_test.out", True)
B_BASIS = "cc-pvdz"
GAMMA = 0.6
bond_length = [2.0]

def do_ct(mol, r, b_basis, gamma):
    mol.r = r
    psi4.set_options({'basis': b_basis,
                      'df_basis_mp2': "cc-pVDZ-F12-Optri",
                      'scf_type': 'pk',
                      'maxiter': 40,
                      'screening': 'csam',
                      'e_convergence': 1e-10})
    e_hf, wfn = psi4.energy("scf", molecule=mol, return_wfn=True)
    mints=MintsHelper(wfn)
    ao_eri=mints.ao_eri().np
    mo_eri=mints.mo_eri(wfn.Ca(),wfn.Ca(),wfn.Ca(),wfn.Ca()).np
    mo_h_core=wfn.Ca().np.T@(mints.ao_potential().np+mints.ao_kinetic().np)@wfn.Ca().np
    V_nuc=psi_mol.nuclear_repulsion_energy()
    n_ele=wfn.nalpha()*2
    n_obs=wfn.nmo()
    write_dump("BARE.DUMP",n_ele=n_ele,n_obs=n_obs,V_nuc=V_nuc,mo_h_core=mo_h_core,mo_eri=mo_eri)
    basis = psi4.core.get_global_option('BASIS')
    df_basis = psi4.core.get_global_option('DF_BASIS_MP2')
    print("regular hf energy is ", e_hf)
    print("run ct scf")
    h_ct = canonical_transform(
        mol, wfn, basis, df_basis, gamma=gamma, frezee_core=True)
    rhf_ct = rhf_energy(psi_mol, wfn, h_ct)
    print("ct  hf energy is ", rhf_ct["escf"],
          " correlation energy is ", rhf_ct["escf"]-e_hf)
    h1e_ct, h2e_ct, cp_ct=convert_ct_to_quccsd(h_ct, rhf_ct)
    h1e_ct_mo=np.einsum("ij,iI,jJ->IJ",h1e_ct,cp_ct,cp_ct,optimize=True)
    h2e_ct_mo=np.einsum("ijkl,iI,jJ,kK,lL->IJKL",h2e_ct,cp_ct,cp_ct,cp_ct,cp_ct,optimize=True)
    write_dump("DRESSED.DUMP",n_ele=n_ele,n_obs=n_obs,V_nuc=V_nuc,mo_h_core=h1e_ct_mo,mo_eri=h2e_ct_mo)
    return e_hf,h_ct, rhf_ct


def convert_ct_to_quccsd(h_ct, hf_ct):
    h1e_ct = np.copy(h_ct['Hbar1'])
    h2e_ct = np.copy(h_ct['Hbar2'])
    h2e_ct = np.einsum("ijkl->ikjl", h2e_ct)  # to ao basis
    cp_ct = hf_ct['C']
    return h1e_ct, h2e_ct, cp_ct



for r_h2 in bond_length:
    print("at bond length {}".format(r_h2))
    psi4.core.clean_options()
    psi4.core.clean()
    psi4.core.clean_variables()
    e_hf,Hct, ct_rhf = do_ct(psi_mol, r_h2, B_BASIS, GAMMA)
    ct_escf = ct_rhf['escf']
    ## save integral
    h1e_ct, h2e_ct, cp_ct=convert_ct_to_quccsd(Hct,ct_rhf)



at bond length 2.0
Molecule: geometry: Molecule is not complete, please use 'update_geometry'
                    once all variables are set.
regular hf energy is  -14.839973000644921
run ct scf
slice(0, 28, None)
(28, 28)
(28, 28, 28, 28)

Number of occupied orbitals: 3
Number of basis functions: 28

  Total time taken for integrals 0.000 seconds.


Total time taken for setup: 0.054 seconds
SCF Iteration   1: Energy = -14.4196457360367134   dE = -1.44196E+01   dRMS = 5.83751E+00
SCF Iteration   2: Energy = -14.8424986011127729   dE = -4.22853E-01   dRMS = 5.24389E+00
SCF Iteration   3: Energy = -14.8552959157053266   dE = -1.27973E-02   dRMS = 5.74963E-01
SCF Iteration   4: Energy = -14.8570323930816670   dE = -1.73648E-03   dRMS = 2.09391E-01
SCF Iteration   5: Energy = -14.8573049823823879   dE = -2.72589E-04   dRMS = 8.30823E-02
SCF Iteration   6: Energy = -14.8573505301145268   dE = -4.55477E-05   dRMS = 3.33610E-02
SCF Iteration   7: Energy = -14.8573584614837486   dE = -7.93137E

In [2]:
r_h2=2.0
print("at bond length {}".format(r_h2))
psi4.core.clean_options()
psi4.core.clean()
psi4.core.clean_variables()
e_hf,Hct, ct_rhf = do_ct(psi_mol, r_h2, B_BASIS, GAMMA)
ct_escf = ct_rhf['escf']
## save integral
h1e_ct, h2e_ct, cp_ct=convert_ct_to_quccsd(Hct,ct_rhf)

at bond length 2.0
regular hf energy is  -14.839973000644921
run ct scf
slice(0, 28, None)
(28, 28)
(28, 28, 28, 28)

Number of occupied orbitals: 3
Number of basis functions: 28

  Total time taken for integrals 0.000 seconds.


Total time taken for setup: 0.001 seconds
SCF Iteration   1: Energy = -14.4196457360367134   dE = -1.44196E+01   dRMS = 5.83751E+00
SCF Iteration   2: Energy = -14.8424986011127729   dE = -4.22853E-01   dRMS = 5.24389E+00
SCF Iteration   3: Energy = -14.8552959157053266   dE = -1.27973E-02   dRMS = 5.74963E-01
SCF Iteration   4: Energy = -14.8570323930816670   dE = -1.73648E-03   dRMS = 2.09391E-01
SCF Iteration   5: Energy = -14.8573049823823879   dE = -2.72589E-04   dRMS = 8.30823E-02
SCF Iteration   6: Energy = -14.8573505301145268   dE = -4.55477E-05   dRMS = 3.33610E-02
SCF Iteration   7: Energy = -14.8573584614837486   dE = -7.93137E-06   dRMS = 1.35181E-02
SCF Iteration   8: Energy = -14.8573598936368860   dE = -1.43215E-06   dRMS = 5.53943E-03
SCF Iter

In [3]:
print(h2e_ct.shape)
perm=np.einsum("ijkl->jikl",h2e_ct)
print(np.linalg.norm(perm-h2e_ct))

(28, 28, 28, 28)
0.14234147128063016


In [4]:
h_sym=(h2e_ct+perm)/2

In [5]:
print(np.allclose(np.einsum("ijkl->jikl",h_sym),h_sym))
print(np.allclose(np.einsum("ijkl->ijkl",h_sym),h_sym))
print(np.allclose(np.einsum("ijkl->ijlk",h_sym),h_sym))
print(np.allclose(np.einsum("ijkl->jilk",h_sym),h_sym))
print(np.allclose(np.einsum("ijkl->klij",h_sym),h_sym))
print(np.allclose(np.einsum("ijkl->klji",h_sym),h_sym))
print(np.allclose(np.einsum("ijkl->lkij",h_sym),h_sym))
print(np.allclose(np.einsum("ijkl->lkji",h_sym),h_sym))

True
True
True
True
True
True
True
True


In [6]:
## here we need to make fci dump into npz
## and read from it 
fcidump={}
fcidump['norb']=1
fcidump['nelec']=1
fcidump['ms2']=0
fcidump['uhf']=False
fcidump['orbsym']=1
fcidump['isym']=1
fcidump['enuc']=0
fcidump['hcore']=np.zeros((10,10))
fcidump['eri']=np.zeros((10,10,10,10))
np.save("fcidump_test",fcidump)

In [9]:
fci_dump_load=np.load("fcidump_test.npy",allow_pickle=True)

In [10]:
fci_dump_load

array({'norb': 1, 'nelec': 1, 'ms2': 0, 'uhf': False, 'orbsym': 1, 'isym': 1, 'enuc': 0, 'hcore': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'eri': array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0.,